In [42]:
import nltk

In [43]:
# nltk.download()

In [44]:
import pandas as pd

In [45]:
#TEST DATA
test = [
   ('The beer was good.', 'pos'),
   ('I do not enjoy my job', 'neg'),
   ("I ain't feeling dandy today.", 'neg'),
   ("I feel amazing!", 'pos'),
   ('Gary is a friend of mine.', 'pos'),
   ("I can't believe I'm doing this.", 'neg')
]

#TRAIN DATA
train = [('I love this sandwich.', 'pos'),
  ('This is an amazing place!', 'pos'),
  ('I feel very good about these beers.', 'pos'),
  ('This is my best work.', 'pos'),
  ("What an awesome view", 'pos'),
  ('I do not like this restaurant', 'neg'),
  ('I am tired of this stuff.', 'neg'),
  ("I can't deal with this", 'neg'),
  ('He is my sworn enemy!', 'neg'),
  ('My boss is horrible.', 'neg')]

In [46]:
train = pd.DataFrame(train, columns=['sentences','sentiments'])
test = pd.DataFrame(test, columns=['sentences','sentiments'])

In [47]:
train

,sentences,sentiments
0,I love this sandwich.,pos
1,This is an amazing place!,pos
2,I feel very good about these beers.,pos
3,This is my best work.,pos
4,What an awesome view,pos
5,I do not like this restaurant,neg
6,I am tired of this stuff.,neg
7,I can't deal with this,neg
8,He is my sworn enemy!,neg
9,My boss is horrible.,neg


In [48]:
from sklearn.preprocessing import LabelEncoder

In [49]:
encoder = LabelEncoder()

In [50]:
train['sentiments'] = encoder.fit_transform(train['sentiments'])
test['sentiments'] = encoder.fit_transform(test['sentiments'])

In [51]:
from nltk.corpus import stopwords
from string import punctuation
stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
lancaster_stemmer = LancasterStemmer()
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

In [52]:
#  nltk.download('stopwords')

In [53]:
train_corpus = train['sentences'].tolist()
test_corpus = test['sentences'].tolist()
train_label = train['sentiments'].tolist()
test_label = test['sentiments'].tolist()

In [54]:
corpus = train_corpus + test_corpus
labels = train_label + test_label

In [55]:
corpus

['I love this sandwich.',
 'This is an amazing place!',
 'I feel very good about these beers.',
 'This is my best work.',
 'What an awesome view',
 'I do not like this restaurant',
 'I am tired of this stuff.',
 "I can't deal with this",
 'He is my sworn enemy!',
 'My boss is horrible.',
 'The beer was good.',
 'I do not enjoy my job',
 "I ain't feeling dandy today.",
 'I feel amazing!',
 'Gary is a friend of mine.',
 "I can't believe I'm doing this."]

In [56]:
final_corpus = []
for i in range(len(corpus)):
    text = word_tokenize(corpus[i].lower())
    text = [lancaster_stemmer.stem(y) for y in text if y not in stuff_to_be_removed]
    sent = " ".join(text)
    final_corpus.append(sent)

In [57]:
# nltk.download('punkt')

In [58]:
labels

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]

In [59]:
df = pd.DataFrame(final_corpus, columns = ['sentences'])

In [60]:
df['sentiments'] = labels

In [61]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [62]:
cv = CountVectorizer()
tfidf = TfidfVectorizer()

In [63]:
vector = tfidf.fit_transform(df['sentences'])

In [64]:
X = pd.DataFrame(vector.toarray(), columns=list(tfidf.get_feature_names()))
y = df['sentiments']

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [66]:
rfc = RandomForestClassifier(random_state=42)

In [67]:
X_train, X_test, y_train, y_test = tts(X,y,test_size = 0.25, random_state= 42)

In [68]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

C:\Users\Ajay\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.25

In [69]:
y_pred = rfc.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       0.00      0.00      0.00         3

    accuracy                           0.25         4
   macro avg       0.12      0.50      0.20         4
weighted avg       0.06      0.25      0.10         4

0.25


C:\Users\Ajay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [70]:
#  Logistic regression
from sklearn.linear_model import LogisticRegression

In [71]:
log_reg = LogisticRegression(random_state=42)

In [72]:
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

C:\Users\Ajay\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [73]:
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

C:\Users\Ajay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       0.00      0.00      0.00         3

    accuracy                           0.25         4
   macro avg       0.12      0.50      0.20         4
weighted avg       0.06      0.25      0.10         4

0.25


In [74]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

In [75]:
nb = MultinomialNB()

In [76]:
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)

In [77]:
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       1.00      0.33      0.50         3

    accuracy                           0.50         4
   macro avg       0.67      0.67      0.50         4
weighted avg       0.83      0.50      0.50         4

0.5


In [78]:
from sklearn.naive_bayes import GaussianNB

In [79]:
gnb = GaussianNB()

In [80]:
gnb.fit(X_train,y_train)
y_pred = gnb.predict(X_test)

In [81]:
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.75      1.00      0.86         3

    accuracy                           0.75         4
   macro avg       0.38      0.50      0.43         4
weighted avg       0.56      0.75      0.64         4

0.75


C:\Users\Ajay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
